In [2]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
x = [[1,2],[3,4],[5,6],[7,8]]
y = [[3],[7],[11],[15]]

In [4]:
class MyDataset(Dataset):
    def __init__(self,x,y):
        self.x = torch.tensor(x).float().to(device)
        self.y = torch.tensor(y).float().to(device)    
    def __getitem__(self,ix):
        return self.x[ix], self.y[ix] #返回（输入x[ix],标签y[ix]）
    def __len__(self):
        return len(self.x)

In [5]:
ds = MyDataset(x,y)
dl = DataLoader(ds, batch_size=2,shuffle=True)

In [6]:
model = nn.Sequential(
     nn.Linear(2,8),
     nn.ReLU(),
     nn.Linear(8,1)
).to(device)

In [7]:
pip install torchsummary

  Using cached torchsummary-1.5.1-py3-none-any.whl (2.8 kB)
Note: you may need to restart the kernel to use updated packages.


In [8]:
from torchsummary import summary

In [9]:
print(summary(model,(2,)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                    [-1, 8]              24
              ReLU-2                    [-1, 8]               0
            Linear-3                    [-1, 1]               9
Total params: 33
Trainable params: 33
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------
None


In [10]:
loss_func = nn.MSELoss()
from torch.optim import SGD
opt = SGD(model.parameters(),lr=0.001)
import time
loss_history = []
start = time.time()
for _ in range(50):
    for ix,iy in dl:
        opt.zero_grad()
        loss_value = loss_func(model(ix),iy)
        loss_value.backward()
        opt.step()
        loss_history.append(loss_value.item())
end = time.time()
print(end - start) #打印总耗时

0.06395101547241211


In [11]:
print(model.state_dict())

OrderedDict([('0.weight', tensor([[ 0.3471, -0.3918],
        [-0.6378, -0.2221],
        [ 0.0506, -0.2210],
        [ 0.1915,  1.2942],
        [-0.4073, -0.0463],
        [ 0.1623, -0.4773],
        [-0.1468, -0.2166],
        [ 0.2640,  0.3018]])), ('0.bias', tensor([ 0.5660,  0.7022,  0.4778, -0.1066,  0.5828,  0.3719, -0.4980,  0.0587])), ('2.weight', tensor([[-0.0772,  0.2010, -0.2740,  1.1190, -0.0141,  0.3084, -0.1200,  0.3216]])), ('2.bias', tensor([0.3614]))])


In [12]:
#模型保存
save_path = 'mymodel.pth' 
torch.save(model.state_dict(), save_path)